In [6]:
import requests
import json
import psycopg2


url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json"
headers = {
    "accept": "application/json",
    "x-api-key": "lCd20Zc6VmbsEyIiMlPso3VOaMzgFxB3wC8k8s3Q"
}
response = requests.get(url, headers=headers)

data =json.loads(response.text)

conn = psycopg2.connect(
    host="localhost",
    database="sports_radar",
    user="postgres",
    password="ASkl1289@"
)
cur = conn.cursor()

ranking_list = []
competitor_list = []

# Loop through all ranking blocks in the list
for ranking in data['rankings']:
    for com in ranking['competitor_rankings']:
        # Extract competitor details
        competitor = com.get('competitor', {})
        country_code = competitor.get('country_code')

        # Skip this entry if country_code is missing (required by DB constraint)
        if not country_code:
            continue

        competitor_id = competitor.get('id')

        competitor_list.append({
            'competitor_id': competitor_id,
            'name': competitor.get('name', 'Unknown'),
            'country': competitor.get('country', 'Unknown'),
            'country_code': country_code,
            'abbreviation': competitor.get('abbreviation', '')
        })

        ranking_list.append({
            'rank': com.get('rank'),
            'movement': com.get('movement'),
            'points': com.get('points'),
            'competitions_played': com.get('competitions_played'),
            'competitor_id': competitor_id  # Link ranking to competitor
        })

print(competitor_list)        

# DB connection and insert
try:
    conn = psycopg2.connect(
        host="localhost",
        database="sports_radar",
        user="postgres",
        password="ASkl1289@"
    )
    cur = conn.cursor()

    # Insert complexes
    for comp in competitor_list:

        cur.execute(
            """
            INSERT INTO sports_radar_schema.competitors (competitor_id, name, country, country_code, abbreviation)
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (competitor_id) DO NOTHING
            """,
            (comp['competitor_id'], comp['name'], comp['country'], comp['country_code'], comp['abbreviation'])
        )

    # Insert venues
    for cr in ranking_list:
        cur.execute(
            """
            INSERT INTO sports_radar_schema.competitor_rankings
            (rank, movement, points, competitions_played, competitor_id)
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (rank_id) DO NOTHING
            """,
            (
                cr['rank'],
                cr['movement'],
                cr['points'],
                cr['competitions_played'],
                cr['competitor_id']
            )
        )

    conn.commit()
    print("Data inserted successfully.")

except Exception as e:
    print("Database operation failed:", e)

finally:
    if conn:
        cur.close()
        conn.close()



[{'competitor_id': 'sr:competitor:49363', 'name': 'Pavic, Mate', 'country': 'Croatia', 'country_code': 'HRV', 'abbreviation': 'PAV'}, {'competitor_id': 'sr:competitor:51836', 'name': 'Arevalo-Gonzalez, Marcelo', 'country': 'El Salvador', 'country_code': 'SLV', 'abbreviation': 'ARE'}, {'competitor_id': 'sr:competitor:14898', 'name': 'Heliovaara, Harri', 'country': 'Finland', 'country_code': 'FIN', 'abbreviation': 'HEL'}, {'competitor_id': 'sr:competitor:637970', 'name': 'Patten, Henry', 'country': 'Great Britain', 'country_code': 'GBR', 'abbreviation': 'PAT'}, {'competitor_id': 'sr:competitor:36593', 'name': 'Krawietz, Kevin', 'country': 'Germany', 'country_code': 'DEU', 'abbreviation': 'KRA'}, {'competitor_id': 'sr:competitor:52293', 'name': 'Putz, Tim', 'country': 'Germany', 'country_code': 'DEU', 'abbreviation': 'PUT'}, {'competitor_id': 'sr:competitor:15568', 'name': 'Granollers, Marcel', 'country': 'Spain', 'country_code': 'ESP', 'abbreviation': 'GRA'}, {'competitor_id': 'sr:compet